# setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
import geopandas as gpd
import xarray as xr
import wget
import requests
import netCDF4 as nc4

In [3]:
date_start= '1981-01-01'
date_end = '2020-12-31'
date_clim_end = '2020-12-31'
lat_min = -25
lat_max = 11
lon_min = -81
lon_max = -34


In [6]:
cru = xr.open_dataset(r'Data\Datasets\raw\cru_ts4.06.1901.2021.pre.dat.nc')
cru['time'] = pd.date_range(start='1901-01-01', end='2021-12-31', freq='MS')
cru = cru.rename({'pre': 'pr'})
cru = cru.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
cru.to_netcdf(r'Data\Datasets\clean\cru.nc')


In [7]:
chirps = xr.open_dataset(r'Data\Datasets\raw\v2p0chirps_mo_25.nc', decode_times=False)
chirps['time'] = pd.date_range(start='1981-01-01', end='2023-04-30', freq='MS')
chirps = chirps.rename({'latitude': 'lat', 'longitude': 'lon'})
chirps = chirps.rename({'precip': 'pr'})
chirps = chirps.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
chirps.to_netcdf(r'Data\Datasets\clean\chirps.nc')




In [9]:
era = xr.open_dataset(r'Data/Datasets/raw/era_land.nc', decode_times=False, chunks='auto')
era['time'] = pd.date_range(start='1950-1-01', end='2023-6-15', freq='MS')
era = era.rename({'tp': 'pr','latitude': 'lat', 'longitude': 'lon'})
era = era.sel(expver=1).drop('expver')
era = era.sortby(era.lat).sel(lat=slice(lat_min, lat_max), time=slice(date_start, date_end))
#change era lon from 0 to 360 to -180 to 180
era = era.assign_coords(lon=(((era.lon + 180) % 360) - 180))#.sortby('lon')
#change era lat to -90, 90
era = era.sel(lon=slice(lon_min, lon_max))
era['pr'] = era['pr'] * (1000 * 30)
era.to_netcdf(r'Data\Datasets\clean\era_land.nc')




c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [10]:
gpcc = xr.open_dataset(r'Data\Datasets\raw\gpcc_05.nc')
gpcc = gpcc.rename({'precip': 'pr'})
#sort lat
gpcc = gpcc.sortby(gpcc.lat)
gpcc = gpcc.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
gpcc.to_netcdf(r'Data\Datasets\clean\gpcc.nc')


In [12]:
terra = xr.open_dataset(r'Data\Datasets\raw\agg_terraclimate_ppt_1958_CurrentYear_GLOBE.nc')
terra = terra.rename({'ppt': 'pr'})
terra = terra.sortby(terra.lat)
terra = terra.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
terra.to_netcdf(r'Data\Datasets\clean\terra.nc')



In [13]:
# Specify the path to the folder containing the NetCDF files
folder_path = r"D:\GU\IC\Scripts\Data\Datasets\raw\xavier"
# Open and concatenate all the NetCDF files
ds = xr.open_mfdataset(os.path.join(folder_path, '*.nc'), combine='by_coords')
# Transform daily data to monthly
ds_monthly = ds.resample(time='1M').mean()
ds_monthly = ds_monthly.rename({'latitude': 'lat', 'longitude': 'lon'})
xavier = ds_monthly.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
xavier ['pr'] = xavier ['pr']*30
xavier.to_netcdf(r'Data\Datasets\clean\xavier.nc')


c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [7]:
# # Specify the path to the folder containing the NetCDF files
#folder_path = r"D:\GU\IC\Scripts\Data\Datasets\raw\chelsa"
#ds = xr.open_mfdataset(os.path.join(folder_path, '*.tif'),combine='nested', concat_dim='month').squeeze()
#ds = ds.rename({'band_data': 'pr', 'x': 'lon', 'y': 'lat'})
##set time coord
#ds['month'] = np.linspace(1, 12, 12)
##sort lat
#ds = ds.sortby(ds.lat)
#ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
#chelsa = ds.drop(['spatial_ref', 'band'])#.to_netcdf(r'Data\Datasets\clean\chelsa.nc')

c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


# gswp

c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


# JRA

In [ ]:
#urls_jra = ['https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1958/fcst_phy2m.061_tprat.reg_tl319.195801_195812',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1959/fcst_phy2m.061_tprat.reg_tl319.195901_195912',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1960/fcst_phy2m.061_tprat.reg_tl319.196001_196012',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1961/fcst_phy2m.061_tprat.reg_tl319.196101_196112',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1962/fcst_phy2m.061_tprat.reg_tl319.196201_196212',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1963/fcst_phy2m.061_tprat.reg_tl319.196301_196312',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1964/fcst_phy2m.061_tprat.reg_tl319.196401_196412',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1965/fcst_phy2m.061_tprat.reg_tl319.196501_196512',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1966/fcst_phy2m.061_tprat.reg_tl319.196601_196612',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1967/fcst_phy2m.061_tprat.reg_tl319.196701_196712',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1968/fcst_phy2m.061_tprat.reg_tl319.196801_196812',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1969/fcst_phy2m.061_tprat.reg_tl319.196901_196912',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1970/fcst_phy2m.061_tprat.reg_tl319.197001_197012',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1971/fcst_phy2m.061_tprat.reg_tl319.197101_197112',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1972/fcst_phy2m.061_tprat.reg_tl319.197201_197212',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1973/fcst_phy2m.061_tprat.reg_tl319.197301_197312',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1974/fcst_phy2m.061_tprat.reg_tl319.197401_197412',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1975/fcst_phy2m.061_tprat.reg_tl319.197501_197512',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1976/fcst_phy2m.061_tprat.reg_tl319.197601_197612',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1977/fcst_phy2m.061_tprat.reg_tl319.197701_197712',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1978/fcst_phy2m.061_tprat.reg_tl319.197801_197812',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1979/fcst_phy2m.061_tprat.reg_tl319.197901_197912',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1980/fcst_phy2m.061_tprat.reg_tl319.198001_198012',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1981/fcst_phy2m.061_tprat.reg_tl319.198101_198112',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1982/fcst_phy2m.061_tprat.reg_tl319.198201_198212',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1983/fcst_phy2m.061_tprat.reg_tl319.198301_198312',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1984/fcst_phy2m.061_tprat.reg_tl319.198401_198412',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1985/fcst_phy2m.061_tprat.reg_tl319.198501_198512',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1986/fcst_phy2m.061_tprat.reg_tl319.198601_198612',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1987/fcst_phy2m.061_tprat.reg_tl319.198701_198712',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1988/fcst_phy2m.061_tprat.reg_tl319.198801_198812',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1989/fcst_phy2m.061_tprat.reg_tl319.198901_198912',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1990/fcst_phy2m.061_tprat.reg_tl319.199001_199012',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1991/fcst_phy2m.061_tprat.reg_tl319.199101_199112',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1992/fcst_phy2m.061_tprat.reg_tl319.199201_199212',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1993/fcst_phy2m.061_tprat.reg_tl319.199301_199312',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1994/fcst_phy2m.061_tprat.reg_tl319.199401_199412',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1995/fcst_phy2m.061_tprat.reg_tl319.199501_199512',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1996/fcst_phy2m.061_tprat.reg_tl319.199601_199612',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1997/fcst_phy2m.061_tprat.reg_tl319.199701_199712',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1998/fcst_phy2m.061_tprat.reg_tl319.199801_199812',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/1999/fcst_phy2m.061_tprat.reg_tl319.199901_199912',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2000/fcst_phy2m.061_tprat.reg_tl319.200001_200012',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2001/fcst_phy2m.061_tprat.reg_tl319.200101_200112',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2002/fcst_phy2m.061_tprat.reg_tl319.200201_200212',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2003/fcst_phy2m.061_tprat.reg_tl319.200301_200312',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2004/fcst_phy2m.061_tprat.reg_tl319.200401_200412',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2005/fcst_phy2m.061_tprat.reg_tl319.200501_200512',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2006/fcst_phy2m.061_tprat.reg_tl319.200601_200612',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2007/fcst_phy2m.061_tprat.reg_tl319.200701_200712',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2008/fcst_phy2m.061_tprat.reg_tl319.200801_200812',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2009/fcst_phy2m.061_tprat.reg_tl319.200901_200912',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2010/fcst_phy2m.061_tprat.reg_tl319.201001_201012',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2011/fcst_phy2m.061_tprat.reg_tl319.201101_201112',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2012/fcst_phy2m.061_tprat.reg_tl319.201201_201212',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2013/fcst_phy2m.061_tprat.reg_tl319.201301_201312',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2014/fcst_phy2m.061_tprat.reg_tl319.201401_201412',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2015/fcst_phy2m.061_tprat.reg_tl319.201501_201512',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2016/fcst_phy2m.061_tprat.reg_tl319.201601_201612',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2017/fcst_phy2m.061_tprat.reg_tl319.201701_201712',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2018/fcst_phy2m.061_tprat.reg_tl319.201801_201812',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2019/fcst_phy2m.061_tprat.reg_tl319.201901_201912',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2020/fcst_phy2m.061_tprat.reg_tl319.202001_202012',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2021/fcst_phy2m.061_tprat.reg_tl319.202101_202112',
'https://data.rda.ucar.edu/ds628.1/fcst_phy2m/2022/fcst_phy2m.061_tprat.reg_tl319.202201_202212']
## Download the file(s)
#for url in urls_jra :
#    output_directory = os.path.join(r"Data\Datasets\raw\JRA", url.split(".")[-1]+'.grib1')
#    filename = wget.download(url, out=output_directory)

In [18]:
folder_path = r"D:\GU\IC\Scripts\Data\Datasets\raw\JRA"
ds = xr.open_mfdataset(os.path.join(folder_path, '*.grib1'),combine='nested', concat_dim='time', engine='cfgrib').squeeze()
ds = ds.rename({'latitude': 'lat', 'longitude': 'lon', 'tpratsfc': 'pr'})
ds = ds.sortby(ds.lat)
ds = ds.drop(['step','surface', 'valid_time']).assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
ds['pr'] = ds['pr']  * 30
ds.to_netcdf(r'Data\Datasets\clean\jra55.nc')


# IMERG

In [20]:
##read the subset txt
#urls = pd.read_csv(r'Data\Datasets\raw\IMERG\subset_GPM_3IMERGM_06_20230711_181340_.txt', sep=" ", header=None)
## transform to string
#urls = urls[0].astype(str)
##exclude first 2 lines
#urls = urls[2:]
#folder_path = r"Data\Datasets\raw\IMERG"
#
##for url in urls:
#    # Set the FILENAME string to the data file name, the LABEL keyword value, or any customized name. 
#    
#    FILENAME = os.path.join(folder_path, url.split(".")[-5]+'.nc4')
#    result = requests.get(url)
#    try:
#        result.raise_for_status()
#        f = open(FILENAME,'wb')
#        f.write(result.content)
#        f.close()
#        print('contents of URL written to '+FILENAME)
#    except:
#        print('requests.get() returned an error code '+str(result.status_code))

In [26]:
folder_path = r"Data\Datasets\raw\IMERG"
ds = xr.open_mfdataset(os.path.join(folder_path, '*.nc4'),combine='nested', concat_dim='time').squeeze()
ds = ds.rename({ 'precipitation': 'pr'})
ds = ds.sortby(ds.lat)
ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
ds['pr'] = ds['pr']  * 30 * 24
ds = ds.transpose('time', 'lat', 'lon')
ds.to_netcdf(r'Data\Datasets\clean\imerg.nc')


# MERRA2

In [22]:
#read the subset txt
#urls = pd.read_csv(r'Data\Datasets\raw\MERRA2\subset_M2TMNXLND_5.12.4_20230711_185833_.txt', sep=" ", header=None)
## transform to string
#urls = urls[0].astype(str)
##exclude first 2 lines
#urls = urls[1:]
#folder_path = r"Data\Datasets\raw\MERRA2"
#
#for url in urls:
#    # Set the FILENAME string to the data file name, the LABEL keyword value, or any customized name. 
#    
#    FILENAME = os.path.join(folder_path, url.split(".")[-3]+'.nc4')
#    result = requests.get(url)
#    try:
#        result.raise_for_status()
#        f = open(FILENAME,'wb')
#        f.write(result.content)
#        f.close()
#        print('contents of URL written to '+FILENAME)
#    except:
#        print('requests.get() returned an error code '+str(result.status_code))

In [25]:
folder_path = r"Data\Datasets\raw\MERRA2"

ds = xr.open_mfdataset(os.path.join(folder_path, '*.nc4'),combine='nested', concat_dim='time').squeeze()
ds = ds.rename({ 'PRECTOTLAND': 'pr'})
ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
ds['pr'] = ds['pr']  * 86400 * 30
ds.to_netcdf(r'Data\Datasets\clean\merra2.nc')


# EXTRAS

In [ ]:
# only 1980-2010.
folder_path = r"D:\GU\IC\Scripts\Data\Datasets\raw\GSWP"
ds = xr.open_mfdataset(os.path.join(folder_path, '*.nc4'),combine='nested', concat_dim='time').squeeze()
ds = ds.sortby(ds.lat)
# Transform daily data to monthly
ds_monthly = ds.resample(time='1M').mean()
gswp = ds_monthly.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
gswp['pr'] = gswp['pr'] * 86400 * 30
gswp.to_netcdf(r'Data\Datasets\clean\gswp3.nc')



In [13]:
era = xr.open_dataset(r'Data\Datasets\raw\era5_tp.nc', decode_times=False)
#create monthly datarange from 1950-1-15  to 2023-4-15
era['time'] = pd.date_range(start='1950-1-01', end='2023-4-15', freq='MS')
era = era.rename({'tp': 'pr'})
#change era lon from 0 to 360 to -180 to 180
era = era.assign_coords(lon=(((era.lon + 180) % 360) - 180))#.sortby('lon')
#change era lat to -90, 90
era = era.sortby(era.lat)
era['pr'] = era['pr']*30
era = era.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
era.to_netcdf(r'Data\Datasets\clean\era.nc')




In [ ]:
#2010-2020 only in v2, 
#cfsr = xr.open_dataset(r'Data\Datasets\raw\ncepcfsr.nc', decode_times=False)
#cfsr['time'] = pd.date_range(start='1979-01-01', end='2010-12-31', freq='MS')
#cfsr = cfsr.rename({'latitude': 'lat', 'longitude': 'lon'})
#cfsr = cfsr.rename({'PRATE_surface': 'pr'})
#cfsr = cfsr.assign_coords(lon=(((cfsr.lon + 180) % 360) - 180)).sortby('lon')
#cfsr = cfsr.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
#cfsr.to_netcdf(r'Data\Datasets\clean\cfsr3.nc')
#



In [20]:

ncep = xr.open_dataset(r'Data\Datasets\raw\ncep2.nc', decode_times=False)
ncep['time'] = pd.date_range(start='1979-01-01', end='2023-05-30', freq='MS')
ncep = ncep.drop('time_bnds').assign_coords(lon=(((ncep.lon + 180) % 360) - 180)).sortby('lon')
ncep = ncep.sortby(ncep.lat)
ncep = ncep.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
ncep = ncep.rename({'cprat': 'pr'})
ncep['pr'] = ncep['pr']  * 86400 * 30
ncep.to_netcdf(r'Data\Datasets\clean\ncep2.nc')


In [54]:
ncep = xr.open_dataset(r'Data\Datasets\raw\ncep1.nc', decode_times=True)
ncep = ncep.assign_coords(lon=(((ncep.lon + 180) % 360) - 180)).sortby('lon')
ncep = ncep.sortby(ncep.lat)
ncep = ncep.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
ncep = ncep.rename({'prate': 'pr'})
ncep['pr'] = ncep['pr']  * 86400 * 30
ncep.to_netcdf(r'Data\Datasets\clean\ncep1.nc')


In [11]:

 # Specify the path to the folder containing the NetCDF files
folder_path = r"D:\GU\IC\Scripts\Data\Datasets\raw\worldclim"
ds = xr.open_mfdataset(os.path.join(folder_path, '*.tif'),combine='nested', concat_dim='time').squeeze()
ds = ds.rename({'band_data': 'pr', 'x': 'lon', 'y': 'lat'})
#set time coord
ds['time'] = pd.date_range(start='1980-01-01', end='2021-12-31', freq='MS')
#sort lat
ds = ds.sortby(ds.lat)
ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
worldclim = ds.drop(['band', 'spatial_ref'])
worldclim.to_netcdf(r'Data\Datasets\clean\worldclim.nc')

c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [8]:
cmap = xr.open_dataset(r'Data\Datasets\raw\cmap.nc')
cmap = cmap.rename({'precip': 'pr'})
cmap = cmap.sortby(cmap.lat)
#fix lon
cmap = cmap.assign_coords(lon=(((cmap.lon + 180) % 360) - 180)).sortby('lon')
cmap = cmap.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
cmap['pr'] = cmap['pr'] * 30
cmap.to_netcdf(r'Data\Datasets\clean\cmap.nc')

## GPCP


In [14]:
##read the subset txt
#urls = pd.read_csv(r'Data\Datasets\raw\GPCP\subset_GPCPMON_3.2_20230921_111931_.txt', sep=" ", header=None)
## transform to string
#urls = urls[0].astype(str)
##exclude first 2 lines
#urls = urls[2:]
#folder_path = r"Data\Datasets\raw\GPCP"
#
#for url in urls:
#    # Set the FILENAME string to the data file name, the LABEL keyword value, or any customized name. 
#    
#    FILENAME = os.path.join(folder_path, url.split(".")[-4].split("/")[-1]+'.nc4')
#    result = requests.get(url)
#    try:
#        result.raise_for_status()
#        f = open(FILENAME,'wb')
#        f.write(result.content)
#        f.close()
#        print('contents of URL written to '+FILENAME)
#    except:
#        print('requests.get() returned an error code '+str(result.status_code))

contents of URL written to Data\Datasets\raw\GPCP\GPCPMON_L3_200706_V3.nc4
contents of URL written to Data\Datasets\raw\GPCP\GPCPMON_L3_200707_V3.nc4
contents of URL written to Data\Datasets\raw\GPCP\GPCPMON_L3_200708_V3.nc4
contents of URL written to Data\Datasets\raw\GPCP\GPCPMON_L3_200709_V3.nc4
contents of URL written to Data\Datasets\raw\GPCP\GPCPMON_L3_200710_V3.nc4


In [29]:
folder_path = r"Data\Datasets\raw\GPCP"
ds = xr.open_mfdataset(os.path.join(folder_path, '*.nc4'),combine='nested', concat_dim='time').squeeze()
ds = ds.rename({ 'sat_gauge_precip': 'pr'})
ds = ds.sortby(ds.lat).sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
ds['pr'] = ds['pr']  * 30
ds.to_netcdf(r'Data\Datasets\clean\gpcp.nc')
#

## PERSIANN

In [34]:
cdr = xr.open_dataset(r'Data\Datasets\raw\CDR_2023-09-25042913pm.nc', decode_times=True)
cdr = cdr.rename({'precip': 'pr', 'datetime': 'time'}).sortby(cdr.lat)
cdr = cdr.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
#drop crs
cdr = cdr.drop('crs')
cdr.to_netcdf(r'Data\Datasets\clean\cdr.nc')

In [20]:
persiann = xr.open_dataset(r'Data\Datasets\raw\PERSIANN_2023-09-25042710pm.nc', decode_times=True)
persiann = persiann.rename({'precip': 'pr', 'datetime': 'time'}).sortby(persiann.lat)
persiann = persiann.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
#drop crs
persiann = persiann.drop('crs')
#persiann.to_netcdf(r'Data\Datasets\clean\persiann.nc')

In [19]:
ccs = xr.open_dataset(r'Data\Datasets\raw\CCS_2023-09-25111254am.nc', decode_times=True)
ccs = ccs.rename({'precip': 'pr', 'datetime': 'time'}).sortby(ccs.lat)
ccs = ccs.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
#drop crs
ccs = ccs.drop('crs')
#to float32
ccs = ccs.astype(np.float32)

ccs.to_netcdf(r'Data\Datasets\clean\ccs.nc')

## CMORPH

In [3]:
#for year in range(1998, 2023):
#    for month in range(1, 13):
#        for day in range(1, 32):
#            url = f"https://www.ncei.noaa.gov/data/cmorph-high-resolution-global-precipitation-estimates/access/daily/0.25deg/{year}/{month:02d}/CMORPH_V1.0_ADJ_0.25deg-DLY_00Z_{year}{month:02d}{day:02d}.nc"
#            FILENAME = os.path.join(r"Data\Datasets\raw\CMORPH", f"{year}{month:02d}{day:02d}.nc")
#            result = requests.get(url)
#            try:
#                result.raise_for_status()
#                f = open(FILENAME,'wb')
#                f.write(result.content)
#                f.close()
#                print('contents of URL written to '+FILENAME)
#            except:
#                print('requests.get() returned an error code '+str(result.status_code))

contents of URL written to Data\Datasets\raw\CMORPH\20060101.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060102.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060103.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060104.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060105.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060106.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060107.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060108.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060109.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060110.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060111.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060112.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060113.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060114.nc
contents of URL written to Data\Datasets\raw\CMORPH\20060115.nc
contents of URL written to Data\Datasets

In [53]:
#folder_path = r"Data\Datasets\raw\CMORPH"
#ds = xr.open_mfdataset(os.path.join(folder_path, '*.nc'),combine='nested', concat_dim='time').squeeze()
#ds = ds[['cmorph']]
#ds = ds.rename({ 'cmorph': 'pr'})
#ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
#ds = ds.sortby(ds.lat).sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
#ds = ds.resample(time='1M').sum()
##ds['pr'] = ds['pr']  * 30
#ds.load()
#ds.to_netcdf(r'Data\Datasets\clean\cmorph.nc')

## CPC

In [4]:
#url = 'https://downloads.psl.noaa.gov/Datasets/cpc_global_precip/precip.1979.nc'
#for year in range(1978, 2023):
#    url = f'https://downloads.psl.noaa.gov/Datasets/cpc_global_precip/precip.{year}.nc'
#    FILENAME = os.path.join(r"Data\Datasets\raw\CPC", f"{year}.nc")
#    result = requests.get(url)
#    try:
#        result.raise_for_status()
#        f = open(FILENAME,'wb')
#        f.write(result.content)
#        f.close()
#        print('contents of URL written to '+FILENAME)
#    except:
#        print('requests.get() returned an error code '+str(result.status_code))

requests.get() returned an error code 404
contents of URL written to Data\Datasets\raw\CPC\1979.nc
contents of URL written to Data\Datasets\raw\CPC\1980.nc
contents of URL written to Data\Datasets\raw\CPC\1981.nc
contents of URL written to Data\Datasets\raw\CPC\1982.nc
contents of URL written to Data\Datasets\raw\CPC\1983.nc
contents of URL written to Data\Datasets\raw\CPC\1984.nc
contents of URL written to Data\Datasets\raw\CPC\1985.nc
contents of URL written to Data\Datasets\raw\CPC\1986.nc
contents of URL written to Data\Datasets\raw\CPC\1987.nc
contents of URL written to Data\Datasets\raw\CPC\1988.nc
contents of URL written to Data\Datasets\raw\CPC\1989.nc
contents of URL written to Data\Datasets\raw\CPC\1990.nc
contents of URL written to Data\Datasets\raw\CPC\1991.nc
contents of URL written to Data\Datasets\raw\CPC\1992.nc
contents of URL written to Data\Datasets\raw\CPC\1993.nc
contents of URL written to Data\Datasets\raw\CPC\1994.nc
contents of URL written to Data\Datasets\raw\C

In [80]:
folder_path = r"Data\Datasets\raw\CPC"
cpc = xr.open_mfdataset(os.path.join(folder_path, '*.nc'),combine='nested', concat_dim='time').squeeze()
cpc = cpc.rename({ 'precip': 'pr'})
cpc = cpc.assign_coords(lon=(((cpc.lon + 180) % 360) - 180)).sortby('lon')
cpc = cpc.sortby(cpc.lat).sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
cpc = cpc.resample(time='1M').mean()
cpc['pr'] = cpc['pr']  * 30
cpc.to_netcdf(r'Data\Datasets\clean\cpc.nc')

c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


## GLDAS 2.1

In [4]:
#read the subset txt
urls = pd.read_csv(r'Data\Datasets\raw\GLDAS_2.1\subset_GLDAS_NOAH025_M_2.1_20230926_121628_.txt', sep=" ", header=None)
# transform to string
urls = urls[0].astype(str)
#exclude first 2 lines
urls = urls[30:]


In [5]:
folder_path = r"Data\Datasets\raw\GLDAS_2.1"

for url in urls:
    # Set the FILENAME string to the data file name, the LABEL keyword value, or any customized name. 
    
    FILENAME = os.path.join(folder_path, url.split(".")[-6]+'.nc')
    result = requests.get(url)
    try:
        result.raise_for_status()
        f = open(FILENAME,'wb')
        f.write(result.content)
        f.close()
        print('contents of URL written to '+FILENAME)
    except:
        print('requests.get() returned an error code '+str(result.status_code))


contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200206.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200207.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200208.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200209.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200210.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200211.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200212.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200301.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200302.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200303.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200304.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200305.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200306.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200307.nc
contents of URL written to Data\Datasets\raw\GLDAS_2.1\A200308.nc
contents o

In [7]:
folder_path = r"Data\Datasets\raw\GLDAS_2.1"
gldas = xr.open_mfdataset(os.path.join(folder_path, '*.nc'),combine='nested', concat_dim='time').squeeze()
gldas = gldas.rename({'Rainf_f_tavg':'pr'})[['pr']]
gldas = gldas.assign_coords(lon=(((gldas.lon + 180) % 360) - 180)).sortby('lon')
gldas = gldas.sortby(gldas.lat).sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
gldas['pr'] = gldas['pr']  * 86400 * 30
gldas.to_netcdf(r'Data\Datasets\clean\gldas.nc')

## MSWEP

In [79]:
folder_path = r"Data\Datasets\raw\MSWEP"
mswep = xr.open_mfdataset(os.path.join(folder_path, '*.nc'),combine='nested', concat_dim='time').squeeze()
mswep = mswep.rename({ 'precipitation': 'pr'})
mswep = mswep.assign_coords(lon=(((mswep.lon + 180) % 360) - 180)).sortby('lon')
mswep = mswep.sortby(mswep.lat).sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
mswep.to_netcdf(r'Data\Datasets\clean\mswep.nc')

c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [81]:
folder_path = r"Data\Datasets\raw\MSWEP_nogauge"
mswep = xr.open_mfdataset(os.path.join(folder_path, '*.nc'),combine='nested', concat_dim='time').squeeze()
mswep = xr.open_mfdataset(os.path.join(folder_path, '*.nc'),combine='nested', concat_dim='time').squeeze()
mswep = mswep.rename({ 'precipitation': 'pr'})
mswep = mswep.assign_coords(lon=(((mswep.lon + 180) % 360) - 180)).sortby('lon')
mswep = mswep.sortby(mswep.lat).sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max), time=slice(date_start, date_end))
mswep.to_netcdf(r'Data\Datasets\clean\mswep_nogauge.nc')

c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
